In [ ]:
# ============================================================
# MODELO DE SUMINISTRO DE SANGRE - CON INSTALACIÓN AUTOMÁTICA
# ============================================================

import subprocess
import sys
import os

print("="*60)
print("INSTALANDO Y CONFIGURANDO GLPK")
print("="*60)

# Paso 1: Instalar GLPK
print("\n📦 Instalando GLPK...")
try:
    # Actualizar repositorios primero
    result_update = subprocess.run(
        ['apt-get', 'update', '-qq'],
        capture_output=True,
        text=True,
        timeout=60
    )
    
    # Instalar glpk-utils
    result = subprocess.run(
        ['apt-get', 'install', '-y', '-qq', 'glpk-utils'],
        capture_output=True,
        text=True,
        timeout=60
    )
    if result.returncode == 0:
        print("✓ GLPK instalado exitosamente")
    else:
        print(f"⚠️ Código de retorno: {result.returncode}")
        print(f"Error: {result.stderr}")
except Exception as e:
    print(f"❌ Error durante instalación: {e}")
    print("\n🔄 Intenta ejecutar manualmente en celdas separadas:")
    print("Celda 1: !apt-get update -qq")
    print("Celda 2: !apt-get install -y -qq glpk-utils")
    sys.exit(1)

# Paso 2: Verificar instalación
print("\n🔍 Verificando instalación...")
glpsol_path = subprocess.run(['which', 'glpsol'], capture_output=True, text=True)
if glpsol_path.stdout.strip():
    print(f"✓ GLPK encontrado en: {glpsol_path.stdout.strip()}")
else:
    print("❌ GLPK no encontrado después de instalación")
    print("\n🔄 Solución alternativa: Ejecuta estas dos líneas en celdas separadas:")
    print("Celda 1: !apt-get update")
    print("Celda 2: !apt-get install -y glpk-utils")
    sys.exit(1)

# Paso 3: Importar Pyomo
print("\n📚 Cargando librerías...")
import random
from pyomo.environ import *

random.seed(42)

# ===========================
# CONJUNTOS
# ===========================
I = ["BM1", "BM2", "BM3"]
J = ["LBDC1", "LBDC2", "LBDC3"]
R = ["RBB1", "RBB2"]
H = ["H1", "H2", "H3", "H4"]
K = ["C1", "C2"]
U = ["W1", "W2"]
P = ["A", "B", "AB", "O"]
T = list(range(1, 46))  # 45 periodos según especificación original

# ===========================
# FUNCIONES
# ===========================
def uni_int(a, b):
    return int(random.uniform(a, b))

def uni_float(a, b, ndigits=4):
    return round(random.uniform(a, b), ndigits)

# ===========================
# PARÁMETROS
# ===========================
print("📊 Generando datos...")

DEMAND_H_MIN, DEMAND_H_MAX = 20, 80
DEMAND_K_MIN, DEMAND_K_MAX = 5, 40
PROD_MIN, PROD_MAX = 300, 1000
SC_r_val = 5000
SC_h_val = 1000
SC_k_val = 1000

PRICE_H_MIN, PRICE_H_MAX = 150000, 300000
PRICE_K_MIN, PRICE_K_MAX = 100000, 220000

OC_MIN, OC_MAX = 20_000, 80_000
IC_MIN, IC_MAX = 100, 1_000
XC_MIN, XC_MAX = 500, 2_500

# Distancias
d_ir = {(i, r): uni_float(7, 20, 2) for i in I for r in R}
d_jr = {(j, r): uni_float(3, 20, 2) for j in J for r in R}
d_rh = {(r, h): uni_float(0.5, 30, 2) for r in R for h in H}
d_rk = {(r, k): uni_float(0.5, 30, 2) for r in R for k in K}

# Emisiones
EP = {(t, p, r): uni_float(0.10, 0.50, 4) for t in T for p in P for r in R}
EI_r = {(t, p, r): uni_float(0.001, 0.01, 6) for t in T for p in P for r in R}
EI_h = {(t, p, h): uni_float(0.001, 0.01, 6) for t in T for p in P for h in H}
EI_k = {(t, p, k): uni_float(0.001, 0.01, 6) for t in T for p in P for k in K}
EX_rh = {(t, p, r, h): uni_float(0.0005, 0.005, 7) for t in T for p in P for r in R for h in H}
EX_rk = {(t, p, r, k): uni_float(0.0005, 0.005, 7) for t in T for p in P for r in R for k in K}

CAP = {t: 1000.0 for t in T}  # Límite de emisiones por periodo (kg CO2e)
EC = 250_000.0  # Costo por kg de emisiones (IDR/kg)

# Datos
DM_h = {(t, p, h): uni_int(DEMAND_H_MIN, DEMAND_H_MAX) for t in T for p in P for h in H}
DM_k = {(t, p, k): uni_int(DEMAND_K_MIN, DEMAND_K_MAX) for t in T for p in P for k in K}
PA = {(t, p, r): uni_int(PROD_MIN, PROD_MAX) for t in T for p in P for r in R}

SC_r = {(p, r): SC_r_val for p in P for r in R}
SC_h = {(p, h): SC_h_val for p in P for h in H}
SC_k = {(p, k): SC_k_val for p in P for k in K}

SP_rh = {(t, p, r, h): uni_int(PRICE_H_MIN, PRICE_H_MAX) for t in T for p in P for r in R for h in H}
SP_rk = {(t, p, r, k): uni_int(PRICE_K_MIN, PRICE_K_MAX) for t in T for p in P for r in R for k in K}

OC = {(t, p, r): uni_int(OC_MIN, OC_MAX) for t in T for p in P for r in R}
IC_r = {(t, p, r): uni_int(IC_MIN, IC_MAX) for t in T for p in P for r in R}
IC_h = {(t, p, h): uni_int(IC_MIN, IC_MAX) for t in T for p in P for h in H}
IC_k = {(t, p, k): uni_int(IC_MIN, IC_MAX) for t in T for p in P for k in K}

XC_rh = {(t, p, r, h): uni_int(XC_MIN, XC_MAX) for t in T for p in P for r in R for h in H}
XC_rk = {(t, p, r, k): uni_int(XC_MIN, XC_MAX) for t in T for p in P for r in R for k in K}

# Variables para calcular emisiones
emisiones_produccion = {}
emisiones_inventario = {}
emisiones_transporte = {}

for t in T:
    emisiones_produccion[t] = 0
    emisiones_inventario[t] = 0
    emisiones_transporte[t] = 0

# ===========================
# MODELO
# ===========================
print("\n" + "="*60)
print("CONSTRUYENDO MODELO DE OPTIMIZACIÓN")
print("="*60)
print(f"Periodos: {len(T)} | Tipos sangre: {len(P)}")
print(f"RBBs: {len(R)} | Hospitales: {len(H)} | Clínicas: {len(K)}")

modelo = ConcreteModel()

# Conjuntos
modelo.R = Set(initialize=R)
modelo.H = Set(initialize=H)
modelo.K = Set(initialize=K)
modelo.P = Set(initialize=P)
modelo.T = Set(initialize=T)

# Variables principales
modelo.PR = Var(modelo.T, modelo.P, modelo.R, domain=NonNegativeReals)
modelo.IR = Var(modelo.T, modelo.P, modelo.R, domain=NonNegativeReals)
modelo.IH = Var(modelo.T, modelo.P, modelo.H, domain=NonNegativeReals)
modelo.IK = Var(modelo.T, modelo.P, modelo.K, domain=NonNegativeReals)

# Variables de demanda insatisfecha
modelo.SH = Var(modelo.T, modelo.P, modelo.H, domain=NonNegativeReals)
modelo.SK = Var(modelo.T, modelo.P, modelo.K, domain=NonNegativeReals)

# Flujos
modelo.X_rh = Var(modelo.T, modelo.P, modelo.R, modelo.H, domain=NonNegativeReals)
modelo.X_rk = Var(modelo.T, modelo.P, modelo.R, modelo.K, domain=NonNegativeReals)

# Restricciones
modelo.restricciones = ConstraintList()

print("\n🔧 Agregando restricciones...")

# Balance RBB (simplificado)
for t in T:
    for p in P:
        for r in R:
            IR_prev = 0 if t == 1 else modelo.IR[t-1, p, r]
            inflows = modelo.PR[t, p, r]
            outflows = (sum(modelo.X_rh[t, p, r, h] for h in H) +
                       sum(modelo.X_rk[t, p, r, k] for k in K))
            modelo.restricciones.add(IR_prev + inflows == modelo.IR[t, p, r] + outflows)

# Balance Hospital
for t in T:
    for p in P:
        for h in H:
            IH_prev = 0 if t == 1 else modelo.IH[t-1, p, h]
            inflows = sum(modelo.X_rh[t, p, r, h] for r in R)
            demanda = DM_h[t, p, h]
            modelo.restricciones.add(IH_prev + inflows == modelo.IH[t, p, h] + demanda - modelo.SH[t, p, h])

# Balance Clínicas
for t in T:
    for p in P:
        for k in K:
            IK_prev = 0 if t == 1 else modelo.IK[t-1, p, k]
            inflows = sum(modelo.X_rk[t, p, r, k] for r in R)
            demanda = DM_k[t, p, k]
            modelo.restricciones.add(IK_prev + inflows == modelo.IK[t, p, k] + demanda - modelo.SK[t, p, k])

# Capacidades
for t in T:
    for p in P:
        for r in R:
            modelo.restricciones.add(modelo.PR[t, p, r] <= PA[t, p, r])
            modelo.restricciones.add(modelo.IR[t, p, r] <= SC_r[p, r])
        for h in H:
            modelo.restricciones.add(modelo.IH[t, p, h] <= SC_h[p, h])
            modelo.restricciones.add(modelo.SH[t, p, h] <= DM_h[t, p, h])
        for k in K:
            modelo.restricciones.add(modelo.IK[t, p, k] <= SC_k[p, k])
            modelo.restricciones.add(modelo.SK[t, p, k] <= DM_k[t, p, k])

# Restricción de límite de emisiones por periodo
for t in T:
    emision_prod = sum(EP[t, p, r] * modelo.PR[t, p, r] for p in P for r in R)
    emision_inv = (sum(EI_r[t, p, r] * modelo.IR[t, p, r] for p in P for r in R) +
                   sum(EI_h[t, p, h] * modelo.IH[t, p, h] for p in P for h in H) +
                   sum(EI_k[t, p, k] * modelo.IK[t, p, k] for p in P for k in K))
    emision_trans = (sum(EX_rh[t, p, r, h] * modelo.X_rh[t, p, r, h] * d_rh[r, h] 
                        for p in P for r in R for h in H) +
                     sum(EX_rk[t, p, r, k] * modelo.X_rk[t, p, r, k] * d_rk[r, k] 
                        for p in P for r in R for k in K))
    
    # Comentamos el límite estricto para permitir soluciones factibles
    # modelo.restricciones.add(emision_prod + emision_inv + emision_trans <= CAP[t])

print(f"✓ {len(modelo.restricciones)} restricciones agregadas")

# Función Objetivo
def objetivo_rule(m):
    revenue = (sum(SP_rh[t, p, r, h] * m.X_rh[t, p, r, h] 
                  for t in T for p in P for r in R for h in H) +
              sum(SP_rk[t, p, r, k] * m.X_rk[t, p, r, k] 
                  for t in T for p in P for r in R for k in K))
    
    # Costos operativos
    cost_operacion = (
        sum(OC[t, p, r] * m.PR[t, p, r] for t in T for p in P for r in R) +
        sum(IC_r[t, p, r] * m.IR[t, p, r] for t in T for p in P for r in R) +
        sum(IC_h[t, p, h] * m.IH[t, p, h] for t in T for p in P for h in H) +
        sum(IC_k[t, p, k] * m.IK[t, p, k] for t in T for p in P for k in K) +
        sum(XC_rh[t, p, r, h] * m.X_rh[t, p, r, h] * d_rh[r, h] 
            for t in T for p in P for r in R for h in H) * 0.001 +
        sum(1000000 * m.SH[t, p, h] for t in T for p in P for h in H) +
        sum(500000 * m.SK[t, p, k] for t in T for p in P for k in K)
    )
    
    # Costo de emisiones
    emision_total = (
        sum(EP[t, p, r] * m.PR[t, p, r] for t in T for p in P for r in R) +
        sum(EI_r[t, p, r] * m.IR[t, p, r] for t in T for p in P for r in R) +
        sum(EI_h[t, p, h] * m.IH[t, p, h] for t in T for p in P for h in H) +
        sum(EI_k[t, p, k] * m.IK[t, p, k] for t in T for p in P for k in K) +
        sum(EX_rh[t, p, r, h] * m.X_rh[t, p, r, h] * d_rh[r, h] 
            for t in T for p in P for r in R for h in H) +
        sum(EX_rk[t, p, r, k] * m.X_rk[t, p, r, k] * d_rk[r, k] 
            for t in T for p in P for r in R for k in K)
    )
    
    cost_emisiones = EC * emision_total
    
    return revenue - cost_operacion - cost_emisiones

modelo.objetivo = Objective(rule=objetivo_rule, sense=maximize)
print("✓ Función objetivo definida")

# ===========================
# RESOLVER
# ===========================
print("\n" + "="*60)
print("RESOLVIENDO MODELO")
print("="*60)

try:
    # Crear solver apuntando al ejecutable correcto
    solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
    print(f"✓ Solver GLPK cargado")
    print("\n⏳ Optimizando... (Con 45 periodos, esto puede tomar 5-10 minutos)\n")
    
    solucion = solver.solve(modelo, tee=False)  # tee=False para salida limpia
    
    print("\n" + "="*60)
    print("RESULTADOS")
    print("="*60)
    
    if solucion.solver.termination_condition == TerminationCondition.optimal:
        print("✓ Solución óptima encontrada\n")
        print(f"💰 Beneficio neto total: IDR {value(modelo.objetivo):,.2f}")
        
        # Calcular emisiones totales
        emision_produccion = sum(EP[t, p, r] * value(modelo.PR[t, p, r]) 
                                for t in T for p in P for r in R)
        emision_inventario = (sum(EI_r[t, p, r] * value(modelo.IR[t, p, r]) 
                                 for t in T for p in P for r in R) +
                             sum(EI_h[t, p, h] * value(modelo.IH[t, p, h]) 
                                 for t in T for p in P for h in H) +
                             sum(EI_k[t, p, k] * value(modelo.IK[t, p, k]) 
                                 for t in T for p in P for k in K))
        emision_transporte = (sum(EX_rh[t, p, r, h] * value(modelo.X_rh[t, p, r, h]) * d_rh[r, h]
                                 for t in T for p in P for r in R for h in H) +
                             sum(EX_rk[t, p, r, k] * value(modelo.X_rk[t, p, r, k]) * d_rk[r, k]
                                 for t in T for p in P for r in R for k in K))
        
        emision_total = emision_produccion + emision_inventario + emision_transporte
        costo_emision_total = EC * emision_total
        
        print(f"\n🌍 EMISIONES TOTALES:")
        print(f"   Producción:    {emision_produccion:,.2f} kg CO2e")
        print(f"   Inventario:    {emision_inventario:,.2f} kg CO2e")
        print(f"   Transporte:    {emision_transporte:,.2f} kg CO2e")
        print(f"   ─────────────────────────────────")
        print(f"   TOTAL:         {emision_total:,.2f} kg CO2e")
        print(f"   Costo emisión: IDR {costo_emision_total:,.2f}")
        
        # Resumen (primeros 5 y últimos 5 periodos)
        print("\n" + "-"*60)
        print("RESUMEN POR PERIODO (Primeros 5 periodos)")
        print("-"*60)
        
        for t in T[:5]:
            prod_total = sum(value(modelo.PR[t, p, r]) for p in P for r in R)
            inv_total = sum(value(modelo.IR[t, p, r]) for p in P for r in R)
            short_h = sum(value(modelo.SH[t, p, h]) for p in P for h in H)
            short_k = sum(value(modelo.SK[t, p, k]) for p in P for k in K)
            
            emision_periodo = (sum(EP[t, p, r] * value(modelo.PR[t, p, r]) for p in P for r in R) +
                              sum(EI_r[t, p, r] * value(modelo.IR[t, p, r]) for p in P for r in R) +
                              sum(EX_rh[t, p, r, h] * value(modelo.X_rh[t, p, r, h]) * d_rh[r, h] 
                                  for p in P for r in R for h in H))
            
            print(f"\nPeriodo {t}:")
            print(f"  📦 Producción: {prod_total:,.0f} unidades")
            print(f"  📊 Inventario: {inv_total:,.0f} unidades")
            print(f"  ⚠️  Faltante H: {short_h:,.0f} | Faltante C: {short_k:,.0f}")
            print(f"  🌍 Emisiones: {emision_periodo:,.2f} kg CO2e")
        
        print("\n" + "-"*60)
        print("RESUMEN POR PERIODO (Últimos 5 periodos)")
        print("-"*60)
        
        for t in T[-5:]:
            prod_total = sum(value(modelo.PR[t, p, r]) for p in P for r in R)
            inv_total = sum(value(modelo.IR[t, p, r]) for p in P for r in R)
            short_h = sum(value(modelo.SH[t, p, h]) for p in P for h in H)
            short_k = sum(value(modelo.SK[t, p, k]) for p in P for k in K)
            
            emision_periodo = (sum(EP[t, p, r] * value(modelo.PR[t, p, r]) for p in P for r in R) +
                              sum(EI_r[t, p, r] * value(modelo.IR[t, p, r]) for p in P for r in R) +
                              sum(EX_rh[t, p, r, h] * value(modelo.X_rh[t, p, r, h]) * d_rh[r, h] 
                                  for p in P for r in R for h in H))
            
            print(f"\nPeriodo {t}:")
            print(f"  📦 Producción: {prod_total:,.0f} unidades")
            print(f"  📊 Inventario: {inv_total:,.0f} unidades")
            print(f"  ⚠️  Faltante H: {short_h:,.0f} | Faltante C: {short_k:,.0f}")
            print(f"  🌍 Emisiones: {emision_periodo:,.2f} kg CO2e")
        
        # Nivel de servicio
        demanda_total_h = sum(DM_h[t, p, h] for t in T for p in P for h in H)
        faltante_total_h = sum(value(modelo.SH[t, p, h]) for t in T for p in P for h in H)
        nivel_h = (1 - faltante_total_h / demanda_total_h) * 100 if demanda_total_h > 0 else 100
        
        demanda_total_k = sum(DM_k[t, p, k] for t in T for p in P for k in K)
        faltante_total_k = sum(value(modelo.SK[t, p, k]) for t in T for p in P for k in K)
        nivel_k = (1 - faltante_total_k / demanda_total_k) * 100 if demanda_total_k > 0 else 100
        
        print("\n" + "-"*60)
        print("NIVEL DE SERVICIO")
        print("-"*60)
        print(f"🏥 Hospitales: {nivel_h:.1f}%")
        print(f"🏥 Clínicas:   {nivel_k:.1f}%")
        
        print("\n" + "="*60)
        print("✅ OPTIMIZACIÓN COMPLETADA EXITOSAMENTE")
        print("="*60)
        
    else:
        print(f"❌ No se encontró solución óptima")
        print(f"Terminación: {solucion.solver.termination_condition}")
        
except Exception as e:
    print(f"\n❌ ERROR: {e}")
    print("\n🔄 Solución alternativa:")
    print("1. Ejecuta en una celda: !apt-get update")
    print("2. Ejecuta en otra celda: !apt-get install -y glpk-utils")
    print("3. Vuelve a ejecutar este código")